# هدف پروژه:
## توضیح کلی پروژه و تعریف صورت مسئله:
در این پروژه، هدف آشنایی با روش‌هایی که برگرفته از طبیعت و انتخاب طبیعی هستند، می‌باشد. در این روش‌ها که بطور کلی الگوریتم‌های ژنتیک نامیده می شوند، ایده‌هایی برای مدل‌سازی جفت‌گیری، جهش و انتخاب طبیعی به کار گرفته می‌شود. در این گونه الگوریتم‌ها، ممکن است با انتخاب معیارهای ساده‌ی انتخاب طبیعی، نتایج مطلوب به دست نیاید و
باید معیاری در نظر گرفته شود که علاوه بر عملکرد فردی، به گوناگونی جمعیت نیز اهمیت دهد.  
الگوریتم‌های ژنتیک عموماً در مسئله‌هایی با فضای حالت بزرگ کاربرد دارند، این الگوریتم‌ها این کار را با نمونه گرفتن از جمعیت و ترکیب و تغییر افراد و ارزیابی آن‌ها انجام می‌دهند و سعی می‌کنند که نسل به نسل جواب‌ها را بهبود دهند تا به جواب مورد نظر برسند.  
در این پروژه با استفاده از الگوریتم‌های ژنتیک، مسئله‌ی رمزگشایی که یکی از مسائل پرکاربرد در حوزه‌ی علوم کامپیوتری می‌باشد پیاده‌سازی شده است.

## مسئله رمزگشایی
هدف از رمزنگاري، تبدیل متن خام (پیام) به متن رمز شده است تا هیچ کس جز مقصد پیام آن را نفهمد. هر چه بازگرداندن متن رمز شده به پیام اصلی از نظر زمانی پیچیده‌تر باشد، رمزنگاری ارزشمندتر است.

## پیاده‌سازی مسئله
در این پروژه یک پیام رمز شده و یک متن مرجع ضمیمه می‌باشد که با استفاده از الگوریتم ژنتیک، کلید مورد نظر را یافته و پیام رمزگشایی خواهد شد.

### بخش صفر : تمیز کردن داده ها و ایجاد لغت نامه
در کلاس Decoder، خصوصیتی به نام dictionary وجود دارد که از آن برای کلماتی که در فایل مرجع وجود دارد استفاده شده است. متادی که وظیفه جمع آوری کلمات را برعهده دارد cleanData است. این متاد از ماژول Regex برای جداسازی حروف انگلیسی از باقی کاراکترها بهره برده است. این متاد دارای دو آرگومان dataset و treshold می‌باشد. چون در متن stop word ها دارای تعداد حروف کمی هستند از این مورد برای شناسایی آن‌ها استفاده شده است. مثلا اگر treshold برابر با 2 باشد کلماتی در dictionary وجود دارند که بیش از دو حرف دارند.  
همانطور که در ابتدا گفته شد، خروجی این متاد نوع داده‌ی dictionary می‌باشد.

In [2]:
def cleanData(self, dataSet, threshold = 0):
    dataSet = re.sub(r'[^A-Za-z]', ' ', dataSet)
    dataSetWords = dataSet.split()
    dataSetWords = [word for word in dataSetWords if len(word) >= threshold]
    dataSetWords = list(dict.fromkeys(dataSetWords))
    return dataSetWords

### بخش یک : مشخص کردن مفاهیم اولیه
مفهومی که برای کروموزوم در نظر گرفته شده ترکیبی از 14 حرف انگلیسی می‌باشد که هر حرف انگلیسی یک ژن است. چون key می‌تواند دارای حروف تکراری باشد بنابراین کروموزوم نیز می‌تواند ژن‌های تکراری داشته باشد.   
تابع random.choice برای ایجاد یک gene و random.choices برای ایجاد یک کروموزوم استفاده شده است. 

In [3]:
def createChromosome(self):
    gnome_len = self.keyLength
    genes = string.ascii_lowercase
    return random.choices(genes, k=gnome_len)

### بخش دو : تولید جمعیت اولیه
به کمک دستور زیر، جمعیتی برابر با POPULATION_SIZE ایجاد خواهد شد که در قسمت های بعد برنامه این جمعیت است که بر روی آن crossover و  mutation انجام می‌شود. 

In [ ]:
population = [self.createChromosome() for i in range(POPULATION_SIZE)]

### بخش سه : پیاده‌سازی و مشخص کردن تابع معیار سازگاری
در متاد fitness کلماتی که خروجی متاد decipherWords می‌باشد با کلمات dictionary مرجع مقایسه شده و تعداد کلمات مشترک شمرده خواهد شد. در واقع کروموزومی بعنوان کروموزم بهتر ارزیابی می‌شود که خروجی fitness آن عدد بزرگتری باشد. به بیان دیگر تعداد تطابق بیشتری بین کلمات خروجی متاد decipherWords و dictionary مرجع وجود داشته باشد.

In [4]:
def fitness(self, chromosome):
    self.decipherWords(chromosome)
    decipheredWords = self.decipheredTextWords

    counter = 0
    for word in decipheredWords:
        if word in self.dictionary:
            counter += 1
    return counter

### بخش چهار : پیاده‌سازی crossover و mutation و تولید جمعیت بعدی
متاد crossover دارای دو کروموزوم والد می‌باشد و در این متاد پس از فراخوانی متاد random با احتمالی رخدادی که قابل تنظیم می‌باشد می‌توان از والد‌ها با احتمال متفاوت ژن اختیار کرد. خروجی این متاد یک کروموزوم است که دارای خصوصیاتی مشابه والدین خود می‌باشد.     
متاد mutation یک کروموزوم را بعنوان ورودی گرفته و با تغییری در ژن‌های آن می‌تواند یک کروموزوم جهش یافته را در خروجی ایجاد نماید.

In [5]:
def crossover(self, father, mother):
    childChromosome = []
    for gp1, gp2 in zip(father, mother):
        prob = random.random()
        if prob < 0.45:
            childChromosome.append(gp1)
        else:
            childChromosome.append(gp2)

    return childChromosome


def mutation(self, chromosome):
    new_chromosome = []
    genes = string.ascii_lowercase
    for gp in chromosome:
        prob = random.random()
        if prob < 0.05 :
            new_chromosome.append(random.choice(genes))
        else:
            new_chromosome.append(gp)

    return new_chromosome

### بخش شش : سوالات
#### سوال 1 )
انتخاب جمعیت خیلی زیاد، زمان اجرای برنامه را طولانی می‌کند. از طرفی با انتخاب جمعیت کم، چون فضای حالت این نوع برنامه‌ها خیلی بزرگ است نسل‌های بعد جمعیت نمی‌توانند تمام فضای حالت را بصورت تصادفی پوشش خوبی بدهند. بنابراین جمعیت در این نوع پروژه‌ها جزء مهمی از hyper parameter ها می‌باشد.

#### سوال 2 )
اگر تعداد جمعیت در هر دوره افزایش یابد، دقت الگوریتم بالا رفته ولی سرعت اجرای آن پایین می‌آید. زیرا سرعت اجرا نسبت معکوسی با تعداد جمعیت کروموزوم‌ها دارد.

#### سوال 3 )
اگر در الگوریتم ژنتیک از crossover استفاده نشود، در حین اجرا نمی‌توان انتظار داشت که دقت الگوریتم در هر تکرار افزایش یابد. از طرفی انتخاب mutation هم خیلی مهم می‌باشد زیرا جهش‌هایی که الگوریتم را وادار می‌کند تا جمعیت تغییر اساسی کند و جهش ژنتیکی در آن اتفاق بیفتد، mutation می‌باشد.

#### سوال 4 )
همانطور که در جواب سوال 3 ذکر شد crossover باعث افزایش دقت خواهد شد و mutation می‌تواند با جهش ژنتیکی سرعت اجرا را بالاتر ببرد.

#### سوال 5 )
crossover می‌تواند در دو، سه یا چند نقطه انجام شود و اگر پس از چند تکرار جمعیت تغییر نکند می‌توان بین crossover هایی که در چند نقطه والدین شکسته می‌شوند یا اینکه از بیش از دو parent استفاده کرد.  
درمورد mutation هم بطور مشابه می‌توان دو، سه یا ژن‌های بیشتری را جهش داد. در الگوریتم مورد استفاده پروژه هم درمورد crossover و هم درمورد mutation شکستن کروموزوم‌ها در چند نقطه انجام شده است.

#### سوال 6 )
نظر شخصی این می‌باشد که Crossover اگر در کنار جمعیت زیاد قرار گیرد می‌تواند موثرتر عمل نماید. درواقع تعداد جمعیت زیاد جایگزینی نه چندان مناسب برای متاد mutation شده است.

#### سوال 7 )
1)  انتخاب مناسب جمعیت راهکاری می‌باشد که می‌تواند سرعت اجرای الگوریتم را بهبود ببخشد.  
2)  تغییر در متاد crosover که شکستن را در چندین نقطه انجام می‌دهد.  
3)  تغییر در برنامه mutation که شکستن در چند نقطه انجام شود.
4)  در شروع الگوریتم stop word ها در نظر گرفته شوند و پس از چند تکرار، بدون در نظر گرفتنشان الگوریتم ادامه پیدا نماید.

### تاثیرانتخاب جمعیت در سرعت اجرای برنامه

|Population  |50       |100      |200      |300      |400      |
|:-----------|:-------:|:-------:|:-------:|:-------:|:-------:|
|Running Time|54.58s   |64.49s   |93.63s   |110.34s  |157.58s  |

### تاثیر نخبه گرایی در سرعت اجرای برنامه

|ELITISM RATE|0.2      |0.3    |0.4     |0.5      |0.6      |
|:-----------|:-------:|:-------:|:-------:|:-------:|:-------:|
|Running Time|54.58s   |54.06s   |42.42s   |35.50s  |48.23s  |

In [1]:
import time
import random, string, re
from operator import itemgetter
import numpy as np

POPULATION_SIZE = 50
ELITISM_RATE = 0.5
CROSSOVER_RATE = 0.9
CONSTASNT = 14
THRESHOLD = 4

class Decoder:

    def __init__(self, globalText, encodedText, keyLength = 14):
        self.threshold = 0
        self.encodedText = encodedText
        self.globalText = globalText
        self.keyLength = keyLength
        self.dictionary = {}
        self.createDictionary()
        self.decipheredTextWords = {}

    def cleanData(self, dataSet, threshold = 0):
        dataSet = re.sub(r'[^A-Za-z]', ' ', dataSet)
        dataSetWords = dataSet.split()
        dataSetWords = [word for word in dataSetWords if len(word) >= threshold]
        dataSetWords = list(dict.fromkeys(dataSetWords))
        return dataSetWords

    def createDictionary(self):
        tmp = self.cleanData(globalText, 0)
        self.dictionary = set(tmp)

    def getDecryptionChar(self, chromosomeChar, cryptionChar):
        cryptionChar = cryptionChar.lower()
        chromosomeChar = chromosomeChar.lower()

        lowercaseChar = string.ascii_lowercase
        length = len(lowercaseChar)
        chromosomeCharIndex = lowercaseChar.find(chromosomeChar)
        cryptionCharIndex = lowercaseChar.find(cryptionChar)
        if(cryptionCharIndex >= chromosomeCharIndex):
            return lowercaseChar[cryptionCharIndex - chromosomeCharIndex]
        else:
            return lowercaseChar[length + cryptionCharIndex - chromosomeCharIndex]

    def getCryptionChar(self, chromosomeChar, char):
        char = char.lower()
        chromosomeChar = chromosomeChar.lower()

        lowercase_char = string.ascii_lowercase
        length = len(lowercase_char)
        return lowercase_char[(lowercase_char.find(chromosomeChar) + lowercase_char.find(char)) % length]

    def maskString(self, str, chromosome):
        mask = []
        index = 0
        for char in str:
            if char in string.ascii_letters:
                mask.append(chromosome[index % self.keyLength])
                index += 1
            else:
                mask.append(' ')
        return mask

    def decipherString(self, chromosome):
        encodedText = self.encodedText
        mask = self.maskString(encodedText, chromosome)
        decipheredChars = []

        for i, char in enumerate(self.encodedText):
            if char in string.ascii_letters:
                decodedChar = self.getDecryptionChar(mask[i], char)
                if char in string.ascii_lowercase:
                    decipheredChars.append(decodedChar.lower())
                elif char in string.ascii_uppercase:
                    decipheredChars.append(decodedChar.upper())
            else:
                decipheredChars.append(char)
        decipheredStr = ""
        return  decipheredStr.join(decipheredChars)

    def decipherWords(self, chromosome):
        self.decipheredTextWords = set(self.cleanData(self.decipherString(chromosome), self.threshold))

    def fitness(self, chromosome):
        self.decipherWords(chromosome)
        decipheredWords = self.decipheredTextWords

        counter = 0
        for word in decipheredWords:
            if word in self.dictionary:
                counter += 1
        return counter

    def createChromosome(self):
        gnome_len = self.keyLength
        genes = string.ascii_lowercase
        return random.choices(genes, k=gnome_len)

    def calculateFitnesses(self, population):
        populationScores = [[self.fitness(population[i]), i] for i in range(len(population))]
        populationScores = sorted(populationScores, key=itemgetter(0), reverse=True)
        return populationScores

    def crossover(self, father, mother):
        childChromosome = []
        for gp1, gp2 in zip(father, mother):
            prob = random.random()
            if prob < 0.45:
                childChromosome.append(gp1)
            else:
                childChromosome.append(gp2)

        return childChromosome

    def mutation(self, chromosome):
        new_chromosome = []
        genes = string.ascii_lowercase
        for gp in chromosome:
            prob = random.random()
            if prob < 0.05 :
                new_chromosome.append(random.choice(genes))
            else:
                new_chromosome.append(gp)

        return new_chromosome

    def geneticAlgorithm(self):
        population = [self.createChromosome() for i in range(POPULATION_SIZE)]

        while True:
            sumOfRanks = len(population) * (len(population) + 1) / 2
            ranksProbabilities = [(i + 1) / sumOfRanks for i in range(len(population))]
            ranksProbabilities = sorted(ranksProbabilities, reverse=True)

            populationScores = self.calculateFitnesses(population)
            self.threshold = min(THRESHOLD, int((THRESHOLD+1) * populationScores[0][0] / len(self.decipheredTextWords)))
            #print(self.threshold)
            chromosome = population[populationScores[0][1]]
            #print(f'{populationScores[0][0]}  {len(self.decipheredTextWords)}   {population[populationScores[0][1]]}')
            if populationScores[0][0] >= len(self.cleanData(self.decipherString(chromosome), THRESHOLD + 1)):

                print("The answer Chromosome is: ")
                print(chromosome, "\n")
                return self.decipherString(chromosome)

            newPopulation = []

            newPopulation.extend(
                [population[populationScores[i][1]] for i in range(int(POPULATION_SIZE * ELITISM_RATE))])

            size = int(CROSSOVER_RATE * POPULATION_SIZE)

            for i in range(size):
                rnd_indices = np.random.choice(len(population), 2, ranksProbabilities)
                parent = [population[i] for i in rnd_indices]

                offspring = self.crossover(parent[0], parent[1])
                newPopulation.append(self.mutation(offspring))

            population = newPopulation

    def decode(self):
        return self.geneticAlgorithm()

if __name__ == '__main__':
    encodedText=open('encoded_text.txt').read()
    globalText=open('global_text.txt').read()
    d=Decoder(globalText,encodedText, keyLength = CONSTASNT)
    
    start = time.time()
    decodedText = d.decode()
    end = time.time()

    print("Time: %s seconds" % (end - start), "\n")
    print("The decoded text is: \n")
    print(decodedText)

The answer Chromosome is: 
['a', 'l', 'b', 'e', 'r', 't', 'e', 'i', 'n', 's', 't', 'e', 'i', 'n'] 

Time: 78.77374267578125 seconds 

The decoded text is: 

Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probableâ€”through the work of Joliot in France as well as Fermi and Szilard in Americaâ€”that it may become possible to set up a nuclear chain rea